### import librairie

In [20]:
import os
import os
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

### Import Dossier

In [3]:
train = r"E:\cours_simplon\data_scientist\Machine_learning\random_forest\exo_2\train-mails"
test = r"E:\cours_simplon\data_scientist\Machine_learning\random_forest\exo_2\test-mails"

### Création fonction pour la création du dico
- A partir des mail

In [4]:
def creation_dico(nom_dossier):
    list_dico = []
    emails = [os.path.join(nom_dossier,f) for f in os.listdir(nom_dossier)]   
    for mail in emails:
        # ouverture lecture des mails
        with open(mail) as m:
            for line in m:
                mots = line.split()
                list_dico += mots
    dico = Counter(list_dico)
    list_to_remove = list(dico)
    
    for item in list_to_remove:
       # on enleve les chiffres
        if item.isalpha() == False:
            del dico[item]
        elif len(item) == 1:
            del dico[item]
    # selectiont des 3000 Mots les plus utilisé 
    dico = dico.most_common(3000)
    return dico

In [6]:
# test fonction
dico = creation_dico(train)

### Creation fonction pour l'extraction des features

In [7]:
def extract_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    train_labels = np.zeros(len(files))
    count = 0;
    docID = 0;
    for fil in files:
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dico):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID,wordID] = words.count(word)
            train_labels[docID] = 0;
            if "spmsg" in fil:
                train_labels[docID] = 1;
                count = count + 1
            docID = docID + 1
    return features_matrix, train_labels

### Création matrix avec les features et labels

In [9]:
features_matrix, labels = extract_features(train)

In [10]:
features_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

#### Creation d'un csv avec notre matrice de features

In [11]:
pd.DataFrame(features_matrix).to_csv("file.csv")

In [15]:
# pareil avec le dossier test
test_feature_matrix, test_labels = extract_features(test)

### Random forest 

In [19]:
model = RandomForestClassifier()
#train model
model.fit(features_matrix, labels)
predicted_labels = model.predict(test_feature_matrix)
print('Score : ', round(accuracy_score(test_labels, predicted_labels),3))

score :  0.973


In [22]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97       130
         1.0       0.96      0.98      0.97       130

    accuracy                           0.97       260
   macro avg       0.97      0.97      0.97       260
weighted avg       0.97      0.97      0.97       260

